In [25]:
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


class PredictionApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Prediction GUI")
        self.data = None
        self.models = {
            "Decision Tree": DecisionTreeRegressor(random_state=0),
            "Random Forest": RandomForestRegressor(random_state=0),
            "Gradient Boosting": GradientBoostingRegressor(random_state=0),
            "XGBoost": XGBRegressor(random_state=0),
        }
        self.selected_model_name = tk.StringVar(value="Select Option")
        self.selected_model = None
        self.scaler_X = MinMaxScaler()
        self.scaler_z = MinMaxScaler()
        self.X_test_scaled = None
        self.z_test = None
        self.z_test_pred = None

        self.prediction_type = tk.StringVar(value="Select Option")  # Default prediction type
        self.input_vars = {}  # Dictionary to store input variables
        self.create_widgets()

    def create_widgets(self):
        # Prediction Type Dropdown
        type_label = tk.Label(self.root, text="Select Prediction Type:")
        type_label.grid(row=0, column=0, padx=10, pady=10)
        type_dropdown = ttk.Combobox(
            self.root, textvariable=self.prediction_type, state="readonly"
        )
        type_dropdown["values"] = [
            "Select Option",
            "Gas Holdup",
            "Mass Transfer Coefficient",
            "Bubble Velocity",
            "Bubble Diameter",
        ]
        type_dropdown.grid(row=0, column=1, padx=10, pady=10)

        # Load Data Button
        load_button = tk.Button(self.root, text="Load Data", command=self.load_data)
        load_button.grid(row=1, column=0, padx=10, pady=10)

        # Model Selection Dropdown
        model_label = tk.Label(self.root, text="Select Model:")
        model_label.grid(row=1, column=1, padx=10, pady=10)
        model_dropdown = ttk.Combobox(
            self.root, textvariable=self.selected_model_name, state="readonly"
        )
        model_dropdown["values"] = ["Select Option"] + list(self.models.keys())
        model_dropdown.grid(row=1, column=2, padx=10, pady=10)
        model_dropdown.bind("<<ComboboxSelected>>", self.update_selected_model)

        # Input Fields for Dimensionless Numbers
        input_frame = tk.LabelFrame(self.root, text="Input Variables")
        input_frame.grid(row=2, column=0, columnspan=3, padx=10, pady=10, sticky="ew")

        input_labels = ["Fr", "Bo", "Ga", "Density Ratio", "Sc"]
        for i, label_text in enumerate(input_labels):
            label = tk.Label(input_frame, text=label_text)
            label.grid(row=i, column=0, padx=10, pady=5, sticky="w")
            entry = tk.Entry(input_frame)
            entry.grid(row=i, column=1, padx=10, pady=5, sticky="w")
            self.input_vars[label_text] = entry

        # Train Model Button
        train_button = tk.Button(self.root, text="Train Model", command=self.train_model)
        train_button.grid(row=3, column=0, padx=10, pady=10)

        # Display Metrics Button
        metrics_button = tk.Button(self.root, text="Display Metrics", command=self.display_metrics)
        metrics_button.grid(row=3, column=1, padx=10, pady=10)

        # Predict Button
        predict_button = tk.Button(self.root, text="Predict", command=self.predict_output)
        predict_button.grid(row=3, column=2, padx=10, pady=10)

    def update_selected_model(self, event=None):
        model_name = self.selected_model_name.get()
        if model_name != "Select Option":
            self.selected_model = self.models[model_name]

    def load_data(self):
        file_path = filedialog.askopenfilename(filetypes=[("Excel files", "*.xlsx")])
        if not file_path:
            return
        try:
            sheet_name_mapping = {
                "Gas Holdup": "alpha",
                "Mass Transfer Coefficient": "kla",
                "Bubble Velocity": "vb",
                "Bubble Diameter": "db",
            }
            sheet_name = sheet_name_mapping.get(self.prediction_type.get())
            if sheet_name is None:
                raise ValueError("Invalid prediction type selected.")
            self.data = pd.read_excel(file_path, sheet_name=sheet_name)
            messagebox.showinfo("Success", f"Data for {self.prediction_type.get()} loaded successfully!")
        except Exception as e:
            messagebox.showerror("Error", f"Failed to load data: {e}")

    def train_model(self):
        if self.data is None:
            messagebox.showerror("Error", "Please load data first.")
            return

        try:
            input_columns = ['Fr', 'Bo', 'Ga', 'Density Ratio', 'Sc']
            if self.prediction_type.get() != "Gas Holdup":
                input_columns.append('Sc')

            X = self.data[input_columns]
            z_column = {
                "Gas Holdup": "alpha",
                "Mass Transfer Coefficient": "kla",
                "Bubble Velocity": "vb",
                "Bubble Diameter": "db",
            }[self.prediction_type.get()]
            z = self.data[[z_column]]

            X_train, X_test, z_train, z_test = train_test_split(X, z, test_size=0.2, random_state=0)
            X_train_scaled = self.scaler_X.fit_transform(X_train)
            self.X_test_scaled = self.scaler_X.transform(X_test)
            z_train_scaled = self.scaler_z.fit_transform(z_train)
            self.z_test = z_test

            self.selected_model.fit(X_train_scaled, z_train_scaled.ravel())
            z_test_pred_scaled = self.selected_model.predict(self.X_test_scaled)
            self.z_test_pred = self.scaler_z.inverse_transform(z_test_pred_scaled.reshape(-1, 1))

            messagebox.showinfo("Success", f"{self.selected_model_name.get()} model trained successfully!")
        except Exception as e:
            messagebox.showerror("Error", f"Failed to train model: {e}")

    def display_metrics(self):
        if self.z_test is None or self.z_test_pred is None:
            messagebox.showerror("Error", "Please train the model first.")
            return

        try:
            test_rmse = np.sqrt(mean_squared_error(self.z_test, self.z_test_pred))
            test_mae = mean_absolute_error(self.z_test, self.z_test_pred)
            test_r2 = r2_score(self.z_test, self.z_test_pred)

            metrics_message = (
                f"Prediction Type: {self.prediction_type.get()}\n"
                f"Model: {self.selected_model_name.get()}\n"
                f"Testing RMSE: {test_rmse:.4f}\n"
                f"Testing MAE: {test_mae:.4f}\n"
                f"Testing R2: {test_r2:.4f}"
            )
            messagebox.showinfo("Model Metrics", metrics_message)
        except Exception as e:
            messagebox.showerror("Error", f"Failed to calculate metrics: {e}")

    def predict_output(self):
        try:
            input_values = [float(entry.get()) if entry.get() else 0.0 for entry in self.input_vars.values()]
            if self.prediction_type.get() != "Gas Holdup" and len(input_values) > 4:
                input_values = input_values[:4]

            input_scaled = self.scaler_X.transform([input_values])
            prediction_scaled = self.selected_model.predict(input_scaled)
            prediction = self.scaler_z.inverse_transform([[prediction_scaled[0]]])
            messagebox.showinfo("Prediction Result", f"{self.prediction_type.get()}: {prediction[0][0]:.4f}")
        except Exception as e:
            messagebox.showerror("Error", f"Failed to predict output: {e}")


if __name__ == "__main__":
    root = tk.Tk()
    app = PredictionApp(root)
    root.mainloop()


In [26]:
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


class PredictionApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Prediction GUI")
        self.data = None
        self.models = {
            "Decision Tree": DecisionTreeRegressor(random_state=0),
            "Random Forest": RandomForestRegressor(random_state=0),
            "Gradient Boosting": GradientBoostingRegressor(random_state=0),
            "XGBoost": XGBRegressor(random_state=0),
        }
        self.selected_model_name = tk.StringVar(value="Select Option")
        self.selected_model = None
        self.scaler_X = MinMaxScaler()
        self.scaler_z = MinMaxScaler()
        self.X_test_scaled = None
        self.z_test = None
        self.z_test_pred = None

        self.prediction_type = tk.StringVar(value="Select Option")  # Default prediction type
        self.input_vars = {}  # Dictionary to store input variables
        self.create_widgets()

    def create_widgets(self):
        # Prediction Type Dropdown
        type_label = tk.Label(self.root, text="Select Prediction Type:")
        type_label.grid(row=0, column=0, padx=10, pady=10)
        type_dropdown = ttk.Combobox(
            self.root, textvariable=self.prediction_type, state="readonly"
        )
        type_dropdown["values"] = [
            "Select Option",
            "Gas Holdup",
            "Mass Transfer Coefficient",
            "Bubble Velocity",
            "Bubble Diameter",
        ]
        type_dropdown.grid(row=0, column=1, padx=10, pady=10)

        # Load Data Button
        load_button = tk.Button(self.root, text="Load Data", command=self.load_data)
        load_button.grid(row=1, column=0, padx=10, pady=10)

        # Model Selection Dropdown
        model_label = tk.Label(self.root, text="Select Model:")
        model_label.grid(row=1, column=1, padx=10, pady=10)
        model_dropdown = ttk.Combobox(
            self.root, textvariable=self.selected_model_name, state="readonly"
        )
        model_dropdown["values"] = ["Select Option"] + list(self.models.keys())
        model_dropdown.grid(row=1, column=2, padx=10, pady=10)
        model_dropdown.bind("<<ComboboxSelected>>", self.update_selected_model)

        # Input Fields for Dimensionless Numbers
        input_frame = tk.LabelFrame(self.root, text="Input Variables")
        input_frame.grid(row=2, column=0, columnspan=3, padx=10, pady=10, sticky="ew")

        input_labels = ["Fr", "Bo", "Ga", "Density Ratio", "Sc"]
        for i, label_text in enumerate(input_labels):
            label = tk.Label(input_frame, text=label_text)
            label.grid(row=i, column=0, padx=10, pady=5, sticky="w")
            entry = tk.Entry(input_frame)
            entry.grid(row=i, column=1, padx=10, pady=5, sticky="w")
            self.input_vars[label_text] = entry

        # Train Model Button
        train_button = tk.Button(self.root, text="Train Model", command=self.train_model)
        train_button.grid(row=3, column=0, padx=10, pady=10)

        # Display Metrics Button
        metrics_button = tk.Button(self.root, text="Display Metrics", command=self.display_metrics)
        metrics_button.grid(row=3, column=1, padx=10, pady=10)

        # Predict Button
        predict_button = tk.Button(self.root, text="Predict", command=self.predict_output)
        predict_button.grid(row=3, column=2, padx=10, pady=10)

    def update_selected_model(self, event=None):
        model_name = self.selected_model_name.get()
        if model_name != "Select Option":
            self.selected_model = self.models[model_name]

    def load_data(self):
        file_path = filedialog.askopenfilename(filetypes=[("Excel files", "*.xlsx")])
        if not file_path:
            return
        try:
            sheet_name_mapping = {
                "Gas Holdup": "alpha",
                "Mass Transfer Coefficient": "kla",
                "Bubble Velocity": "vb",
                "Bubble Diameter": "db",
            }
            sheet_name = sheet_name_mapping.get(self.prediction_type.get())
            if sheet_name is None:
                raise ValueError("Invalid prediction type selected.")
            self.data = pd.read_excel(file_path, sheet_name=sheet_name)
            messagebox.showinfo("Success", f"Data for {self.prediction_type.get()} loaded successfully!")
        except Exception as e:
            messagebox.showerror("Error", f"Failed to load data: {e}")

    def train_model(self):
        if self.data is None:
            messagebox.showerror("Error", "Please load data first.")
            return

        try:
            # Determine input columns based on prediction type
            input_columns = ['Fr', 'Bo', 'Ga', 'Density Ratio']
            if self.prediction_type.get() == "Mass Transfer Coefficient":
                input_columns.append('Sc')

            X = self.data[input_columns]
            z_column = {
                "Gas Holdup": "alpha",
                "Mass Transfer Coefficient": "kla",
                "Bubble Velocity": "vb",
                "Bubble Diameter": "db",
            }[self.prediction_type.get()]
            z = self.data[[z_column]]

            X_train, X_test, z_train, z_test = train_test_split(X, z, test_size=0.2, random_state=0)
            X_train_scaled = self.scaler_X.fit_transform(X_train)
            self.X_test_scaled = self.scaler_X.transform(X_test)
            z_train_scaled = self.scaler_z.fit_transform(z_train)
            self.z_test = z_test

            self.selected_model.fit(X_train_scaled, z_train_scaled.ravel())
            z_test_pred_scaled = self.selected_model.predict(self.X_test_scaled)
            self.z_test_pred = self.scaler_z.inverse_transform(z_test_pred_scaled.reshape(-1, 1))

            messagebox.showinfo("Success", f"{self.selected_model_name.get()} model trained successfully!")
        except Exception as e:
            messagebox.showerror("Error", f"Failed to train model: {e}")

    def display_metrics(self):
        if self.z_test is None or self.z_test_pred is None:
            messagebox.showerror("Error", "Please train the model first.")
            return

        try:
            test_rmse = np.sqrt(mean_squared_error(self.z_test, self.z_test_pred))
            test_mae = mean_absolute_error(self.z_test, self.z_test_pred)
            test_r2 = r2_score(self.z_test, self.z_test_pred)

            metrics_message = (
                f"Prediction Type: {self.prediction_type.get()}\n"
                f"Model: {self.selected_model_name.get()}\n"
                f"Testing RMSE: {test_rmse:.4f}\n"
                f"Testing MAE: {test_mae:.4f}\n"
                f"Testing R2: {test_r2:.4f}"
            )
            messagebox.showinfo("Model Metrics", metrics_message)
        except Exception as e:
            messagebox.showerror("Error", f"Failed to calculate metrics: {e}")

    def predict_output(self):
        try:
            input_values = [float(entry.get()) if entry.get() else 0.0 for entry in self.input_vars.values()]
            if self.prediction_type.get() != "Mass Transfer Coefficient":
                input_values = input_values[:4]

            input_scaled = self.scaler_X.transform([input_values])
            prediction_scaled = self.selected_model.predict(input_scaled)
            prediction = self.scaler_z.inverse_transform([[prediction_scaled[0]]])
            messagebox.showinfo("Prediction Result", f"{self.prediction_type.get()}: {prediction[0][0]:.4f}")
        except Exception as e:
            messagebox.showerror("Error", f"Failed to predict output: {e}")


if __name__ == "__main__":
    root = tk.Tk()
    app = PredictionApp(root)
    root.mainloop()


C:\Users\hp\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
C:\Users\hp\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
